# BERT 多类微调（Colab 快速启动）

本笔记本用于在 Google Colab 上使用 GPU 训练多类标签模型。

步骤概览：
1. 安装依赖并克隆仓库
2. 上传三份多类数据集 CSV 到 `data/processed/`
3. 运行训练脚本（使用 `label_multi_cls` 标签列）
4. 查看指标并打包下载模型输出

提示：Runtime → Change runtime type → Hardware accelerator 选择 GPU。

In [ ]:
# 安装依赖（transformers/datasets/accelerate/evaluate/scikit-learn）
!pip -q install -U transformers datasets accelerate evaluate scikit-learn

In [ ]:
# 克隆仓库并进入目录
!git clone https://github.com/Caria-Tarnished/Graduation_Project.git -b main
%cd Graduation_Project

In [ ]:
# 挂载 Google Drive，并设置数据与模型目录（建议使用此方式管理数据与权重）
from google.colab import drive
import os

drive.mount('/content/drive')

DRIVE_ROOT = '/content/drive/MyDrive'
DATA_DIR = f'{DRIVE_ROOT}/datasets/xauusd_multilabel'  # 你可以根据需要修改路径
MODEL_DIR = f'{DRIVE_ROOT}/models/bert_xauusd_multilabel_6cls'

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

print('DATA_DIR =', DATA_DIR)
print('MODEL_DIR =', MODEL_DIR)

In [ ]:
# 配置预训练模型（FinBERT）
# 默认使用英文金融领域 FinBERT。如需中文金融领域模型，请将 MODEL_NAME 改为合适的中文 FinBERT 检查点。
MODEL_NAME = 'hfl/chinese-roberta-wwm-ext'  # 可改为中文金融BERT检查点，例如（示例占位，按需替换）
print('MODEL_NAME =', MODEL_NAME)

In [ ]:
# 基于 Drive 中的数据进行基本检查（6类，不做过滤）
import pandas as pd
import os

files = ['train_multi_labeled.csv','val_multi_labeled.csv','test_multi_labeled.csv']
for name in files:
    path = f'{DATA_DIR}/{name}'
    if not os.path.exists(path):
        print('缺少文件：', path)
        continue
    df = pd.read_csv(path, encoding='utf-8')
    if 'label_multi_cls' not in df.columns:
        raise ValueError('数据集缺少 label_multi_cls 列：'+path)
    vc = df['label_multi_cls'].value_counts().sort_index()
    print('\n', name, '标签分布(6类)：')
    print(vc.to_dict())

print('\n6 类数据集检查完成。')

In [ ]:
# 将已上传到仓库目录的数据集复制到 Google Drive（若存在则覆盖）
import os, shutil
repo_proc = '/content/Graduation_Project/data/processed'
files = ['train_multi_labeled.csv','val_multi_labeled.csv','test_multi_labeled.csv']
for name in files:
    src_repo = f'{repo_proc}/{name}'
    dst_drive = f'{DATA_DIR}/{name}'
    if os.path.exists(src_repo):
        shutil.copy2(src_repo, dst_drive)
        print('Copied local -> Drive:', src_repo, '->', dst_drive)
    elif os.path.exists(dst_drive):
        print('Found on Drive:', dst_drive)
    else:
        print('Missing both local and Drive:', name, '请使用 files.upload 上传到 DATA_DIR')
print('DATA_DIR 内容:')
print(os.listdir(DATA_DIR))

In [ ]:
# 上传三份多类训练集到 data/processed/
import os
os.makedirs('data/processed', exist_ok=True)
from google.colab import files
for name in ['train_multi_labeled.csv','val_multi_labeled.csv','test_multi_labeled.csv']:
    print('请上传:', name)
    up = files.upload()  # 在弹窗中选择对应文件
    fname = list(up.keys())[0]
    os.replace(fname, f'data/processed/{name}')
print('上传完成')

In [ ]:
# 将关键信息注入环境变量，便于在 shell 命令中引用
import os
os.environ['DATA_DIR'] = DATA_DIR
os.environ['MODEL_DIR'] = MODEL_DIR
os.environ['MODEL_NAME'] = MODEL_NAME
print('Set ENV:')
print('  DATA_DIR =', os.environ['DATA_DIR'])
print('  MODEL_DIR =', os.environ['MODEL_DIR'])
print('  MODEL_NAME =', os.environ['MODEL_NAME'])

In [ ]:
# 使用 FinBERT + 6分类，从 Google Drive 读取数据，并将模型保存到 Google Drive（改进版：类权重/早停/更久训练）
!python scripts/modeling/bert_finetune_cls.py \
  --train_csv "$DATA_DIR/train_multi_labeled.csv" \
  --val_csv   "$DATA_DIR/val_multi_labeled.csv" \
  --test_csv  "$DATA_DIR/test_multi_labeled.csv" \
  --output_dir "$MODEL_DIR" \
  --label_col label_multi_cls \
  --model_name "$MODEL_NAME" \
  --train_bs 16 --eval_bs 32 \
  --epochs 5 --lr 1e-5 --max_length 384 \
  --class_weight auto \
  --warmup_ratio 0.06 --weight_decay 0.01 \
  --eval_steps 100 --save_steps 100 --early_stopping_patience 2 \
  --gradient_accumulation_steps 1

In [ ]:
# 查看指标（val/test）
import json, os

dir_path = MODEL_DIR  # 使用上面设置的 Drive 输出目录
for nm in ['metrics_val.json','metrics_test.json']:
    p = os.path.join(dir_path, nm)
    if os.path.exists(p):
        with open(p, 'r', encoding='utf-8') as f:
            print(nm, json.load(f))
    else:
        print('missing:', p)

In [ ]:
# 打包并下载输出目录（Drive 中的 6 类模型）
import shutil, os
from google.colab import files

out_dir = MODEL_DIR  # Drive 上的输出目录
zip_base = 'bert_xauusd_multilabel_6cls'
zip_path = f'{zip_base}.zip'

if os.path.isdir(out_dir):
    if os.path.exists(zip_path):
        os.remove(zip_path)
    shutil.make_archive(zip_base, 'zip', out_dir)
    files.download(zip_path)
else:
    print('输出目录不存在：', out_dir)